In [176]:
#making file from video id
# type below with vidoe_id 
video_id = 252984523 #sample vid
#'twitch-chatlog {video_id} -r -no-progress --end=2018-07-20T20:37:44+0000 > {output_filename}'
# if with color; 
# 'twitch-chatlog {video_id} -c -r -no-progress --end=2018-07-20T20:37:44+0000 > {output_filename}'
import json
from dateutil import parser
import datetime

In [ ]:
def open_video_file(filename):
    with open('data/'+filename,'r') as f:
        data = json.load(f)
    return data    
data = open_video_file(video_id)

In [4]:
#helper function
def jprint(j):
    print(json.dumps(j,indent=4,sort_keys=True))

In [233]:
parser.parse('2h30m32s') - parser.parse('0h0m0s')

datetime.timedelta(0, 9032)

In [ ]:
def get_client_id():
    with open('clientid.txt','r') as f:
        cid= f.readline().strip()
    return cid

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
def get_video_lengths(vids):
    url = 'https://api.twitch.tv/helix/videos'
    params = {'id': vids } #['252984523','252984524','257570102']}
    headers = {'Client-ID': get_client_id()} #JM id
    page = requests.get(url, headers=headers,params=params)
    soup = bs(page.text,'lxml')
    videos = json.loads(soup.select('p')[0].text)['data']
    vlens = []
    for v in videos:
        vlens.append(parser.parse(v['duration'])- parser.parse('0h0m0s'))
    return vlens

def get_video_length(vid):
    first = data[0]
    last = data[-1]
    deltime = parser.parse(last['created_at'])-parser.parse(first['created_at'])
    return deltime

In [239]:
print((get_video_length(data)))
print(parser.parse(data[-1]['created_at']))
print(get_video_lengths(['252984523'])[0])

2:45:03.593000
2018-04-21 23:26:53.416000+00:00
<html><body><p>{"data":[{"id":"252984523","user_id":"102381201","title":"C9 따효니 오늘 박제각 잡음","description":"","created_at":"2018-04-21T20:41:41Z","published_at":"2018-04-21T20:41:41Z","url":"https://www.twitch.tv/videos/252984523","thumbnail_url":"https://static-cdn.jtvnw.net/s3_vods/30e04568c0447767a274_ddahyoni_28419446736_844885585/thumb/thumb0-%{width}x%{height}.jpg","viewable":"public","view_count":4983,"language":"ko","type":"archive","duration":"2h44m52s"}],"pagination":{}}</p></body></html>
2:44:52


In [91]:
#interval should be second
def cdf(data,interval):
    counting_ = {}
    last_time = data[-1]['content_offset_seconds']
    n = int(last_time/interval)
    for i in range(n+1):
        counting_[(i*interval,(i+1)*interval)] = 0
    for log in data:
        t = log['content_offset_seconds']
        p = (int(t/interval)*interval,(int(t/interval)+1)*interval)
        counting_[p]+=1
    return counting_
        
        
        

In [94]:
cdf_video= cdf(data,60)

In [101]:
def get_chatrate(data):
    vlen = get_video_length(data)
    m= vlen.seconds//60
    return len(data)/m

In [102]:
get_chatrate(data) # chat per min

90.32727272727273

In [118]:
def get_chatlen_cdf(data):
    d= {}
    for log in data:
        mlen = len(log['message']['body'])
        d[mlen] = d.get(mlen,0)+1
    return d

In [119]:
cdf_chatlen = get_chatlen_cdf(data)

In [124]:
with open('chatlen.csv','w') as f:
    for k in cdf_chatlen.keys():
        f.write(str(k)+','+str(cdf_chatlen[k])+'\n')

In [127]:
with open('chat_cdf.csv','w') as f:
    for k in cdf_video.keys():
        f.write(str(k[0])+','+str(cdf_video[k])+'\n')

In [132]:
def user_chatfreq(data):
    res = {}
    for log in data:
        res[log['commenter']['_id']] =res.get(log['commenter']['_id'],0)+1
    return res

In [133]:
user_chatfreq_dict = user_chatfreq(data)

In [134]:
len(user_chatfreq_dict.keys())

1711

In [136]:
with open('user_chat.csv','w') as f:
    for k in user_chatfreq_dict.keys():
        f.write(str(k)+','+str(user_chatfreq_dict[k])+'\n')

In [145]:
def user_chat(data):
    res = {}
    for log in data:
        t = res.get(log['commenter']['_id'],[])
        t.append(log['message']['body'])
        res[log['commenter']['_id']] = t
    return res

In [146]:
uc = user_chat(data)

In [158]:
uc['163024752']

['따하',
 'BibleThump BibleThump BibleThump BibleThump',
 'TPFufun',
 '잠크리트 인구측정 ㄷㄷㄷ',
 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
 '따즈',
 'ㄹㄹㅇ',
 '??',
 '노래방 디스코1',
 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
 '무슨일이지~',
 '바하',
 '그래 5 좋아',
 '스펀지밥 ㄷㄷ',
 'ResidentSleeper ResidentSleeper',
 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
 '?',
 '건전지 충전하러감',
 'BloodTrail BloodTrail',
 'BloodTrail BloodTrail BloodTrail',
 'BloodTrail BloodTrail',
 'ㅋㅋㅋㅋㅋㅋㅋㅋ',
 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
 '패배의 기운이 스멀스멀',
 '상특',
 'ㅋㅋㅋㅋㅋㅋㅋ',
 '킹룡의갓집 나와주세요',
 'ㄱㄱㄱㄱㅇㄱㅇ',
 '리로이',
 '105등각',
 'ㅇㅎㅎ',
 '상대는 12시다',
 '누가 새우탕에 약넣엇냐',
 '따라닥스',
 'ㅈㅈ',
 '님도 성기사하셈ㅋ',
 '살아남지 못하리라~',
 '?',
 'ㅁㅊㄷㅁㅊㅇ',
 '갓을방학',
 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
 'naga',
 '띠링',
 '줄건줘',
 '따라닥스',
 '띠링도적',
 '들썩',
 '바하',
 '????',
 '6코 적 하수인에게 랜덤8딜쓰냐',
 '기도메타',
 '리로이뿐이야',
 '나와드렷읍니다',
 '들썩',
 'PogChamp PogChamp',
 '부 ㅡ 엉',
 'Kreygasm',
 ':/ PogChamp PogChamp']

In [174]:
def get_video_json(video_id):
    import subprocess
    endpoint = '2018-07-20T20:37:44+0000'
    #video_id = 252984523
    output = '{video_id}.json'.format(video_id=video_id)
    cmd = 'twitch-chatlog {video_id} -r -no-progress --end={endpoint} > data/{output_filename}'\
    .format(video_id=video_id,endpoint=endpoint,output_filename=output)
    subprocess.call(cmd,shell=True)
    return 

In [246]:
def main():
    import pandas as pd
    filename= 'asjdkfl' #vid list filename
    #vids = pvd(filename) # output = list of video id
    vids = ['252984523','257570102']#['252984523']
    
    lst = [['video_id','video_length','chat_count','chat_avg_len','chat_cnt_per_person','chat_per_min','total_chatter']]
    v_lengths = get_video_lengths(vids)
    
    for i in range(len(vids)):
        #basic setting
        vid = vids[i]
        #generate {vid}.json
        get_video_json(vid)
        #1 video id
        lst_v = [vid]
        filename = '{vid}.json'.format(vid=vid)
        # open file as data
        data = open_video_file(filename)
        #2 video length
        lst_v.append(v_lengths[i])
        #3 chat count
        chat_cnt = len(data)
        lst_v.append(chat_cnt)
        #4 chat_avg_len  = total char of chat / chat count
        uc = get_chatlen_cdf(data)
        s = 0
        for k in uc.keys():
            s +=k*uc[k]
        cal = s/chat_cnt
        lst_v.append(cal)
        #5 chat_cnt_per_person = chat count / total_chatter 
        usr_chat =user_chatfreq(data)
        total_chatter = len(list(usr_chat.keys()))
        ccpp = chat_cnt/total_chatter
        lst_v.append(ccpp)
        #6 chat_per_min = chat count / video length
        lst_v.append(get_chatrate(data))
        #7 total_chatter 
        lst_v.append(total_chatter)
        lst.append(lst_v)
    #saving as csv    
    df = pd.DataFrame(lst[1:],columns=lst[0])# output_data[0]
    df.to_csv('summary_statstics_testing.csv')
    return
        
        

In [247]:
main()

<html><body><p>{"data":[{"id":"252984523","user_id":"102381201","title":"C9 따효니 오늘 박제각 잡음","description":"","created_at":"2018-04-21T20:41:41Z","published_at":"2018-04-21T20:41:41Z","url":"https://www.twitch.tv/videos/252984523","thumbnail_url":"https://static-cdn.jtvnw.net/s3_vods/30e04568c0447767a274_ddahyoni_28419446736_844885585/thumb/thumb0-%{width}x%{height}.jpg","viewable":"public","view_count":4983,"language":"ko","type":"archive","duration":"2h44m52s"},{"id":"257570102","user_id":"102381201","title":"C9 따효니","description":"","created_at":"2018-05-04T11:54:40Z","published_at":"2018-05-04T11:54:40Z","url":"https://www.twitch.tv/videos/257570102","thumbnail_url":"https://static-cdn.jtvnw.net/s3_vods/9796a81d156eb265c4a8_ddahyoni_28570383584_854319138/thumb/thumb0-%{width}x%{height}.jpg","viewable":"public","view_count":3590,"language":"ko","type":"archive","duration":"10h21m56s"}],"pagination":{}}</p></body></html>


In [216]:
soup.select('p')[0]

<p>{"data":[{"id":"252984523","user_id":"102381201","title":"C9 따효니 오늘 박제각 잡음","description":"","created_at":"2018-04-21T20:41:41Z","published_at":"2018-04-21T20:41:41Z","url":"https://www.twitch.tv/videos/252984523","thumbnail_url":"https://static-cdn.jtvnw.net/s3_vods/30e04568c0447767a274_ddahyoni_28419446736_844885585/thumb/thumb0-%{width}x%{height}.jpg","viewable":"public","view_count":4983,"language":"ko","type":"archive","duration":"2h44m52s"},{"id":"252984524","user_id":"186616900","title":"PUBG WackyJacky101 birthday server duos","description":"","created_at":"2018-04-21T20:41:41Z","published_at":"2018-04-21T20:41:41Z","url":"https://www.twitch.tv/videos/252984524","thumbnail_url":"https://static-cdn.jtvnw.net/s3_vods/96772d66686b2b82163f_konfuzer17_28419446496_844885570/thumb/thumb0-%{width}x%{height}.jpg","viewable":"public","view_count":0,"language":"en","type":"archive","duration":"35m33s"}],"pagination":{}}</p>

In [225]:
d[0]

{'created_at': '2018-04-21T20:41:41Z',
 'description': '',
 'duration': '2h44m52s',
 'id': '252984523',
 'language': 'ko',
 'published_at': '2018-04-21T20:41:41Z',
 'thumbnail_url': 'https://static-cdn.jtvnw.net/s3_vods/30e04568c0447767a274_ddahyoni_28419446736_844885585/thumb/thumb0-%{width}x%{height}.jpg',
 'title': 'C9 따효니 오늘 박제각 잡음',
 'type': 'archive',
 'url': 'https://www.twitch.tv/videos/252984523',
 'user_id': '102381201',
 'view_count': 4983,
 'viewable': 'public'}